## 필요한 데이터 불러오기 및 접근

## 필요한 라이브러리 설치

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/nlp_liflow/

/content/drive/My Drive/nlp_liflow


In [3]:
!ls

자연어처리_1.ipynb		  ko.bin
자연어처리_2.ipynb		  list_to_csv.csv
all_df_with_tokens.csv		  nlp_tokenizer.ipynb
all_sen.csv			  스포츠_sen.csv
cc.ko.300.bin.gz		  sen_preprocessig.csv
df_all				  스포츠_sen_prerprocessing.csv
df_all.csv			  sentence_Summarizer.ipynb
df_all_GloVe.csv		  textrank
df_all_tfidf.csv		  textrank_git
feature.pkl			  tfidftransformer.pkl
feature.pkl.mtx			  자연어처리_tomotopy_HDPModel.ipynb
first_embedding_clustering.model  자연어처리_tomotopy.ipynb
glove_model.model		  자연어처리_tomotopy_modeling.ipynb
glove_word_dict_128.pickle	  vectorizer.pkl
item_1.csv			  word2vec_model
item_all.csv


In [4]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-11-21 03:26:31--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-11-21 03:26:32--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2d16bf8002887387c26376de3c.dl.dropboxusercontent.com/cd/0/get/BxIzUspXwc4WUyGTPjAQ_9S1I2k6xtzx3lAqt3wN8cNLiOcr4ulV8kmDZS_TGKhSka07C-lk_A-ttHxOW_qNUUUh0umcGRFSiYPom_2QgZkb5W-kp0s9WPE4R7yXoih595i95R69Ckpy7R0DbX0PZryykGZjHgQJHGVaM5z7SHknXlXGnQ3eVR-uEbMWRe-_4SE/file?dl=1# [following]
--2022-11-21 03:26:32--  https://uc2d16bf8002887387c26376de3c.dl.dropboxusercontent.com/cd/0/get/BxIzUspXw

In [5]:
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 2.3 MB/s 
     |████████████████████████████████| 131 kB 32.4 MB/s 
  Created wheel for kss: filename=kss-3.6.4-py3-none-any.whl size=42448613 sha256=5b3e9e7aa29e5657e85143b2f4c8d2776d055b14adf6a5d8a0db7634a5c5b087
  Stored in directory: /root/.cache/pip/wheels/32/53/7c/76bdf098c2a6c5cd4c4c29648da30ad4793d604314b2aeb26f
Successfully built kss


In [6]:
!pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 4.0 MB/s 


In [32]:
!pip install koalanlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!pip install Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import textrank_git
from textrank_git import rank
from textrank_git import sentence
from textrank_git import summarizer
from textrank_git import utils
from textrank_git import word
import re
import kss
import pandas as pd
from konlpy.tag import Mecab
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim import models
import gensim
import logging
import os
from collections import namedtuple, defaultdict
from collections.abc import Iterable
from timeit import default_timer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from numpy import zeros, float32 as REAL, vstack, integer, dtype
import numpy as np
from gensim import utils, matutils  # utility fnc for pickling, common scipy operations etc
from gensim.utils import deprecated
from gensim.models import Word2Vec # noqa: F401
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

## 원본 대화(스포츠)의 키워드 추출하기
key_textrank는 원문 키워드를 말하며

해당 과정은 textrank를 사용해 추출되었습니다.

이 작업은 추후에 doc2vec을 사용해 군집분석을 진행할 때 사용될 단어 리스트를 추리는 작업입니다.

In [39]:
import csv
from koalanlp import API
from koalanlp.proc import SentenceSplitter
from koalanlp.Util import initialize

#initialize(okt='LATEST')

splitter = SentenceSplitter(API.OKT)

def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]
    
key_textrank = []
sep_text = []
with open('/content/drive/MyDrive/nlp_liflow/sen_preprocessig.csv', 'r', encoding='utf-8') as f:
    a = csv.reader(f)
    list_of_csv = list(a)
    #print(list_of_csv)
    for i in list_of_csv:
      #print(i[1][2:-2])
      '''
      if len(i[1][2:-2]) > 1000:
        #print(i[1][2:-2])
        #k = splitter(i[1][2:-2])
        #print(k)
        list_chunked = list_chunk(k, int(len(k)*0.1))
        for l in list_chunked:
          k = "".join(list_chunked)
          key_textrank.append(k)
      else:
        '''
      key_textrank.append(i[1][2:-2])
key_textrank


['nten',
 '감탄을 금치 못할 이야기도 나오고 그래 또 나는 축구에 대해 얘기하고 싶네참 오래된 얘기지만 과거에 서독 프로 축구에 차범근 선수가 나왔잖아그때 우리나라를 위해서 열심히 뛰고 또 후배 양성도 많이 했잖아그러니까 우리 후배들이 또 열심히 해서 정말 좋은 선수들이 줄줄이 나왔지자네는 우리나라 축구 선수들을 이렇게 보면 어떤 생각이 드는가 차범근이 후배들을 많이 키워서 우리나라에 정말 좋은 선수들이 많이 나왔지 요새는 최고의 스타가 손흥민이네그리고 옛날에 박지성도 있고 이영표도 있지 그다음에 여러 후배 선수들이 차범근 씨로 인해서 많이 나왔잖아그리고 또 외국 무대에서 뛸 수 있는 그런 조건을 만들어 주기도 했고 지금 손흥민이 세계에서 상위 몇 번째 안에 드는 선수라고도 그러잖아그리고 또 이십세 미만에서 나온 선수 있지 이강인 선수라든가 이런 선수들이 또 세계적인 선수가 되려고 하고 있잖아요 돈도 엄청 벌어요걔네들이 또 우리나라에 돈을 많이 벌어다 갖다 주는 것 같거든 그러다가 이번에 또 선수로 발탁돼서 선수 몇 명이 외국으로 나가고 나가면 그런 선수들은 또 외화를 많이 벌어들이지 엄청 벌어들이고 있잖아그래서 내 생각에는 우리나라가 어린 선수들을 빨리 성장시키고 했으면 좋겠어그래서 외국으로 많이 나갈 수 있도록 국가 시책으로 그런 걸 하면 좋겠네그래서 청소년들을 키우는 그런 방법을 모의해 봤으면 좋겠다는 생각이야자네 의견은 또 어떤지 모르겠네자네 의견이 어떤지 한번 들어 보자고 나는 이천 이년 월드컵 때 세계적인 선수가 나올 거라고 믿어 의심치 않았어우리나라에서 명장 히딩크 감독을 초빙해 가지고 꿈에 그리던 사강에 진출하고 그런 광경을 볼 때 정말 우리 축구가 앞으로 세계 정상에 설 수 있겠다 싶고 참 우리가 얼마나 그 많은 사람들 속에서 즐겁게 놀고 응원하고 했냐고 광주뿐만 아니고 전국이 떠들썩하도록 떠내려갈 정도로 힘차게 응원했잖아 나는 세계적인 강호 이태리를 격파했을 때 얼마나 흥분했는지 몰라전국이 그냥 흥분의 도가니 속으로 들어가가지고 난리

In [40]:

mecab = Mecab()
stopwords = ['우리']
def tokenizer(raw, pos=["NNG","NNP","VV","VA"], stopword=stopwords): 
  return [word for word, tag in mecab.pos(raw) if len(word) > 1 and tag in pos and word not in stopword ]
key_textrank = key_textrank[1::]

In [41]:
token = []
for i in key_textrank:
  token.append(tokenizer(i))

In [42]:
token

[['감탄',
  '이야기',
  '나오',
  '축구',
  '얘기',
  '얘기',
  '과거',
  '서독',
  '프로',
  '축구',
  '차범근',
  '선수',
  '그때',
  '우리나라',
  '후배',
  '양성',
  '후배',
  '선수',
  '나라',
  '축구',
  '선수',
  '생각',
  '차범근',
  '후배',
  '나라',
  '선수',
  '요새',
  '최고',
  '스타',
  '손흥민',
  '옛날',
  '박지성',
  '이영표',
  '다음',
  '후배',
  '선수',
  '차범근',
  '외국',
  '무대',
  '조건',
  '만들',
  '손흥민',
  '세계',
  '상위',
  '선수',
  '그러',
  '미만',
  '선수',
  '이강인',
  '선수',
  '선수',
  '세계',
  '선수',
  '나라',
  '그러',
  '이번',
  '선수',
  '발탁',
  '선수',
  '외국',
  '나가',
  '선수',
  '외화',
  '벌어들이',
  '벌어들이',
  '생각',
  '나라',
  '선수',
  '성장',
  '외국',
  '국가',
  '시책',
  '청소년',
  '키우',
  '방법',
  '모의',
  '생각',
  '의견',
  '모르',
  '의견',
  '월드컵',
  '세계',
  '선수',
  '의심',
  '나라',
  '명장',
  '히딩크',
  '감독',
  '초빙',
  '그리',
  '사강',
  '진출',
  '광경',
  '축구',
  '세계',
  '정상',
  '사람',
  '즐겁',
  '응원',
  '광주',
  '전국',
  '정도',
  '힘차',
  '응원',
  '세계',
  '강호',
  '이태리',
  '격파',
  '흥분',
  '전국',
  '흥분',
  '도가니',
  '난리'],
 ['강도',
  '직전',
  '주민',
  '센터',
  '이용',
  '다음',
  '스쿼시',
  '다음',
  '수영',
  

In [43]:
import pickle
import re
import numpy as np
from scipy.sparse.dia import dia_matrix
import joblib
from glove import Glove
from glove import Corpus

In [44]:
corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(token, window=5)
glove = Glove(no_components=2, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [45]:
glove.save("glove_model.model")

In [46]:
glove_model = Glove.load('glove_model.model')

In [47]:
# word dict 생성
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

# save word_dict
with open('glove_word_dict_128.pickle', 'wb') as f:
    pickle.dump(word_dict, f)
print('[Success !] Save word dict!...')

[Success !] Lengh of word dict... :  2228
[Success !] Save word dict!...


In [48]:
def sent2vec_glove(tokens, embedding_dim=2):
    '''문장 token 리스트를 받아서 임베딩 시킨다.'''
  
    size = len(tokens)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(tokens):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

In [49]:
matrix = sent2vec_glove(token)

In [50]:
doc_vec = pd.DataFrame(columns=['convrs','trained_vector','token'])

for text, i, t in zip(key_textrank,matrix,token):
  new_row = pd.DataFrame([[text,i,t]],columns =  doc_vec.columns)
  doc_vec = pd.concat([doc_vec, new_row], ignore_index = True)
  print(f"text: {text}, tokens: {t}")
  print(f"trained_doc_vec: {i}")

text: 감탄을 금치 못할 이야기도 나오고 그래 또 나는 축구에 대해 얘기하고 싶네참 오래된 얘기지만 과거에 서독 프로 축구에 차범근 선수가 나왔잖아그때 우리나라를 위해서 열심히 뛰고 또 후배 양성도 많이 했잖아그러니까 우리 후배들이 또 열심히 해서 정말 좋은 선수들이 줄줄이 나왔지자네는 우리나라 축구 선수들을 이렇게 보면 어떤 생각이 드는가 차범근이 후배들을 많이 키워서 우리나라에 정말 좋은 선수들이 많이 나왔지 요새는 최고의 스타가 손흥민이네그리고 옛날에 박지성도 있고 이영표도 있지 그다음에 여러 후배 선수들이 차범근 씨로 인해서 많이 나왔잖아그리고 또 외국 무대에서 뛸 수 있는 그런 조건을 만들어 주기도 했고 지금 손흥민이 세계에서 상위 몇 번째 안에 드는 선수라고도 그러잖아그리고 또 이십세 미만에서 나온 선수 있지 이강인 선수라든가 이런 선수들이 또 세계적인 선수가 되려고 하고 있잖아요 돈도 엄청 벌어요걔네들이 또 우리나라에 돈을 많이 벌어다 갖다 주는 것 같거든 그러다가 이번에 또 선수로 발탁돼서 선수 몇 명이 외국으로 나가고 나가면 그런 선수들은 또 외화를 많이 벌어들이지 엄청 벌어들이고 있잖아그래서 내 생각에는 우리나라가 어린 선수들을 빨리 성장시키고 했으면 좋겠어그래서 외국으로 많이 나갈 수 있도록 국가 시책으로 그런 걸 하면 좋겠네그래서 청소년들을 키우는 그런 방법을 모의해 봤으면 좋겠다는 생각이야자네 의견은 또 어떤지 모르겠네자네 의견이 어떤지 한번 들어 보자고 나는 이천 이년 월드컵 때 세계적인 선수가 나올 거라고 믿어 의심치 않았어우리나라에서 명장 히딩크 감독을 초빙해 가지고 꿈에 그리던 사강에 진출하고 그런 광경을 볼 때 정말 우리 축구가 앞으로 세계 정상에 설 수 있겠다 싶고 참 우리가 얼마나 그 많은 사람들 속에서 즐겁게 놀고 응원하고 했냐고 광주뿐만 아니고 전국이 떠들썩하도록 떠내려갈 정도로 힘차게 응원했잖아 나는 세계적인 강호 이태리를 격파했을 때 얼마나 흥분했는지 몰라전국이 그냥 흥분의 도가니 속으로 들어가가지고 난리도 아니었

In [58]:
doc_vec

,convrs,trained_vector,token
0,감탄을 금치 못할 이야기도 나오고 그래 또 나는 축구에 대해 얘기하고 싶네참 오래된...,"[0.17302857227405302, 0.16147248295567493]","[감탄, 이야기, 나오, 축구, 얘기, 얘기, 과거, 서독, 프로, 축구, 차범근,..."
1,강도도 제일 센 걸로 했는데 위에서 이렇게 딱 꺼내 보니까 직전이야 주민센터 그쪽에...,"[0.11902009719215816, 0.0439116641595429]","[강도, 직전, 주민, 센터, 이용, 다음, 스쿼시, 다음, 수영, 순위, 자전거,..."
2,같이 일선발이 나와서 이렇게 게임을 했는데 일회 하고 이회 시작할 때 비가 엄청 오...,"[0.1353836779376909, 0.11906813071551772]","[선발, 게임, 이회, 시작, 평소, 기아, 타이거즈, 그날, 하루, 즐겁, 하루,..."
3,거의 십년 전이니까 그 당시만해도 골프가 대중적인 스포츠는 아니었으요 그래서 몇 명...,"[0.2680082575387454, 0.21672888669213958]","[당시, 골프, 대중, 스포츠, 조금, 마음, 상황, 계룡산, 근처, 탁구, 어렵,..."
4,골프 치려면은 또 돈이 많아야 되는데 한국에 돈 많은 사람이 되게 많은가 보다 싶기...,"[0.20618640889784703, 0.1630155285319035]","[골프, 한국, 사람, 만들, 한골, 성장, 나라, 여자, 수구, 나라, 중학생, ..."
...,...,...,...
223,한시간쯤 지나니까 얼굴이 빨개져서 들어왔네그러더니 스쿼트를 했대그러면 땀으로 목욕을...,"[0.1906443964568452, 0.16698185960063736]","[시간, 지나, 얼굴, 쿼트, 목욕, 해외여행, 마음, 요가, 힘들, 도전, 욕심,..."
224,할 줄 아는 것이 일단은 그거잖아우리 나이도 있어서 오래는 못 하지근데 사실 나이를...,"[0.41665816918786097, 0.2515150444755045]","[나이, 오래, 나이, 수록, 운동, 다치, 친구, 동창, 그러, 얘기, 친구, 골..."
225,항상 운동 전에 수분 섭취를 해야돼그래서 나는 러닝머신 스피드를 육 점 영 정도 놓...,"[0.30443623096104777, 0.1787556935003639]","[운동, 수분, 섭취, 러닝, 머신, 스피드, 정도, 힘들, 시작, 스로, 기구, ..."
226,해 보니까 근육이 파열돼서 피가 나오는 바람에 고생하고 있거든그러면 고전 무용 말고...,"[0.28571294091078403, 0.15060195885321165]","[근육, 파열, 나오, 고생, 고전, 무용, 수영, 한국, 무용, 운동, 무리, 첫..."


## PCA

In [ ]:
feature=[]
for i in range(0,10):
    a = "feature"+str(i)
    feature.append(a)
# 1500 개의 주성분으로 차원 축소
pca = PCA(n_components= 10) 
printcipalComponents = pca.fit_transform(matrix)

principalDf = pd.DataFrame(data=printcipalComponents, columns = feature)

In [ ]:
pca.explained_variance_ratio_

array([9.89662262e-01, 6.96105469e-03, 1.51995227e-03, 2.94066421e-04,
       2.01813475e-04, 1.35189325e-04, 1.26401445e-04, 1.01007762e-04,
       8.55376393e-05, 5.91485671e-05])

In [ ]:
sum(pca.explained_variance_ratio_)

0.9991464340543519

In [ ]:
# 2차원으로 축소
n_components = 2

# t-SNE 모델 생성
tsne = TSNE(n_components = n_components)

# 학습한 결과 2차원 공간값 출력
tsnedata = tsne.fit_transform(principalDf)

print(tsnedata)
column = ['x', 'y']
tsnedata = pd.DataFrame(tsnedata, columns = column)

doc_vec['tsne_x'] = tsnedata['x']
doc_vec['tsne_y'] = tsnedata['y']

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[[ 1.69359818e+01 -5.12909472e-01]
 [ 2.05818768e+01  1.08454657e+00]
 [ 2.16490173e+01  3.13658565e-01]
 [-9.24236488e+00  2.84833884e+00]
 [ 1.19697313e+01 -3.08985170e-02]
 [ 6.87359381e+00  1.24974281e-01]
 [ 2.31441097e+01  7.61751652e-01]
 [ 2.39315243e+01  6.73990846e-01]
 [ 1.56372948e+01 -4.77502972e-01]
 [-1.95160580e+01  9.00760353e-01]
 [-9.87336063e+00  3.28766155e+00]
 [ 2.11259766e+01  1.16146469e+00]
 [-1.04854221e+01  1.60281253e+00]
 [ 1.21672764e+01 -5.26388109e-01]
 [ 5.13766098e+00 -6.89930856e-01]
 [-2.01999912e+01  1.94150543e+00]
 [-1.99295101e+01  3.51509511e-01]
 [ 3.78426969e-01  1.73917460e+00]
 [ 7.19197941e+00  4.40652043e-01]
 [-4.66153669e+00  1.39966559e+00]
 [-1.98655586e+01  1.71092141e+00]
 [ 2.38699818e+01  3.16658378e-01]
 [-1.62703266e+01 -2.43781544e-02]
 [-1.69609470e+01  1.73265815e-01]
 [-2.43324814e+01  1.15514481e+00]
 [-3.95076537e+00 -5.11218429e-01]
 [ 2.95339394e+00  6.08789206e-01]
 [-2.53685951e+00  7.61220336e-01]
 [ 1.70343685e+01  6

In [ ]:
doc_vec

,convrs,trained_vector,token,tsne_x,tsne_y
0,감탄을 금치 못할 이야기도 나오고 그래 또 나는 축구에 대해 얘기하고 싶네참 오래된...,"[-0.013580725481325497, -0.008895837647219546,...","[감탄, 이야기, 나오, 축구, 얘기, 얘기, 과거, 서독, 프로, 축구, 차범근,...",16.935982,-0.512909
1,강도도 제일 센 걸로 했는데 위에서 이렇게 딱 꺼내 보니까 직전이야 주민센터 그쪽에...,"[-0.0076811790735826415, -0.01370158858787008,...","[강도, 직전, 주민, 센터, 이용, 다음, 스쿼시, 다음, 수영, 순위, 자전거,...",20.581877,1.084547
2,같이 일선발이 나와서 이렇게 게임을 했는데 일회 하고 이회 시작할 때 비가 엄청 오...,"[-0.008392819686019943, -0.00965636376071651, ...","[선발, 게임, 이회, 시작, 평소, 기아, 타이거즈, 그날, 하루, 즐겁, 하루,...",21.649017,0.313659
3,거의 십년 전이니까 그 당시만해도 골프가 대중적인 스포츠는 아니었으요 그래서 몇 명...,"[-0.01645794845427679, -0.03232966393764214, 0...","[당시, 골프, 대중, 스포츠, 조금, 마음, 상황, 계룡산, 근처, 탁구, 어렵,...",-9.242365,2.848339
4,골프 치려면은 또 돈이 많아야 되는데 한국에 돈 많은 사람이 되게 많은가 보다 싶기...,"[-0.012302859418537624, -0.016829139234253648,...","[골프, 한국, 사람, 만들, 한골, 성장, 나라, 여자, 수구, 나라, 중학생, ...",11.969731,-0.030899
...,...,...,...,...,...
223,한시간쯤 지나니까 얼굴이 빨개져서 들어왔네그러더니 스쿼트를 했대그러면 땀으로 목욕을...,"[-0.01210011631449749, -0.024612092805931313, ...","[시간, 지나, 얼굴, 쿼트, 목욕, 해외여행, 마음, 요가, 힘들, 도전, 욕심,...",8.006122,1.842651
224,할 줄 아는 것이 일단은 그거잖아우리 나이도 있어서 오래는 못 하지근데 사실 나이를...,"[-0.02543177225685491, -0.04835681803933757, 0...","[나이, 오래, 나이, 수록, 운동, 다치, 친구, 동창, 그러, 얘기, 친구, 골...",-20.763773,1.461078
225,항상 운동 전에 수분 섭취를 해야돼그래서 나는 러닝머신 스피드를 육 점 영 정도 놓...,"[-0.018114193944004493, -0.03315875845838446, ...","[운동, 수분, 섭취, 러닝, 머신, 스피드, 정도, 힘들, 시작, 스로, 기구, ...",-9.139541,0.487378
226,해 보니까 근육이 파열돼서 피가 나오는 바람에 고생하고 있거든그러면 고전 무용 말고...,"[-0.018347298948159074, -0.035791100772219565,...","[근육, 파열, 나오, 고생, 고전, 무용, 수영, 한국, 무용, 운동, 무리, 첫...",-10.358855,-0.084532


In [60]:
doc_vec['trained_vector']

0      [0.17302857227405302, 0.16147248295567493]
1       [0.11902009719215816, 0.0439116641595429]
2       [0.1353836779376909, 0.11906813071551772]
3       [0.2680082575387454, 0.21672888669213958]
4       [0.20618640889784703, 0.1630155285319035]
                          ...                    
223     [0.1906443964568452, 0.16698185960063736]
224     [0.41665816918786097, 0.2515150444755045]
225     [0.30443623096104777, 0.1787556935003639]
226    [0.28571294091078403, 0.15060195885321165]
227     [0.14478851023669218, 0.1424146541817476]
Name: trained_vector, Length: 228, dtype: object

## 군집화 진행해보기
시도해 볼 cluster 수는 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20 개 입니다.

### Elbow curve 
Cluster 간의 거리의 합을 나타내는 inertia가 급격히 떨어지는 구간이 생기는데 이 구간을 확인해 최적 군집 개수를 구하고자 합니다.


### silhouette score
군집 타당성 지표인 silhouette score를 사용해 적정 군집을 확인해 볼 예정입니다.

### Elbow curve


In [59]:
print("##"*30)
print("K-Means Clustering")
print("##"*30)
# 시도해 볼 cluster 수
cluster_number = range(5,101)
distortions = []
#processed_data = tsnedata.copy()
scaler = preprocessing.MinMaxScaler()
scaler.fit_transform(doc_vec['trained_vector'])
#scaler.fit_transform(processed_data[['x', 'y']])

# 모델 학습
for k in cluster_number:
    Clustering_Method = KMeans(n_clusters=k, random_state = 0)
    #X = model.docvecs.vectors_docs 
    Clustering_Method.fit(doc_vec['trained_vector'])
    distortions.append(Clustering_Method.inertia_)
    
# Elbow Curve 시각화
fig = plt.figure(figsize=(15, 5))
plt.plot(cluster_number, distortions)
plt.grid(True)
plt.title('Elbow curve')
plt.show()

############################################################
K-Means Clustering
############################################################


ValueError: ignored

### K_means 사용한 clustering 결과

In [ ]:
from math import*
def euclidean_distance(x, y):
    return sqrt(sum(pow(a - b, 2) for a, b in zip(x, y)))

In [ ]:
doc_vec['trained_vector'][0]

array([-1.35807255e-02, -8.89583765e-03,  2.10340363e-02,  3.25669504e-02,
       -7.87353200e-03, -2.05825589e-02, -1.76193536e-02, -1.86583341e-02,
       -2.17678800e-02,  1.34139237e-02, -3.52542457e-02, -1.83672723e-02,
       -2.36195421e-02, -8.26210527e-03,  1.54645681e-02, -3.48202896e-02,
        2.81407311e-02,  3.59343060e-03,  7.14253318e-03, -1.99100972e-03,
        1.23384438e-02, -2.61639415e-02,  1.47136857e-02, -4.88603765e-02,
       -9.70375967e-03,  1.90299458e-02, -1.35037954e-02,  2.93088337e-02,
        2.81366576e-02, -4.50790940e-02, -2.57781092e-02,  1.00610493e-02,
       -9.86347675e-03, -1.84171394e-02,  2.34976910e-03,  1.26633709e-02,
       -5.40416547e-03, -8.00876369e-03, -4.42255715e-03, -1.00747552e-03,
        3.36080846e-02,  3.44712928e-04, -3.05445789e-02, -9.16432452e-03,
       -1.21345691e-02,  6.42206505e-03,  2.76052035e-02,  6.28265153e-04,
       -2.37023067e-02, -1.48285159e-02, -2.34792486e-02, -7.63206184e-02,
        1.47712878e-02,  

In [ ]:
Clustering_Method.labels_

array([2, 7, 7, 8, 4, 4, 7, 7, 2, 3, 8, 7, 8, 4, 0, 3, 3, 0, 4, 0, 3, 7,
       3, 3, 1, 0, 0, 0, 2, 7, 0, 4, 6, 3, 2, 5, 4, 3, 0, 3, 8, 2, 4, 3,
       3, 7, 0, 2, 3, 0, 7, 3, 0, 3, 3, 4, 2, 8, 7, 4, 4, 4, 0, 3, 8, 0,
       8, 0, 8, 8, 4, 2, 3, 0, 0, 0, 4, 4, 9, 0, 0, 7, 2, 4, 8, 1, 0, 8,
       0, 0, 8, 4, 7, 2, 0, 8, 8, 8, 0, 4, 3, 8, 8, 3, 6, 2, 7, 4, 7, 9,
       0, 8, 8, 6, 8, 8, 2, 2, 2, 3, 8, 4, 3, 0, 2, 4, 0, 7, 0, 0, 4, 4,
       3, 3, 6, 6, 4, 4, 8, 4, 6, 7, 6, 0, 2, 7, 0, 1, 8, 0, 9, 7, 2, 0,
       8, 4, 8, 0, 8, 3, 2, 4, 2, 6, 3, 8, 7, 0, 8, 9, 7, 3, 3, 2, 4, 4,
       8, 3, 0, 0, 0, 4, 7, 4, 1, 4, 3, 8, 8, 2, 7, 2, 0, 8, 0, 0, 4, 3,
       8, 4, 3, 6, 0, 0, 4, 4, 8, 0, 0, 4, 4, 8, 7, 4, 8, 2, 9, 2, 8, 0,
       7, 6, 0, 4, 6, 8, 8, 7], dtype=int32)

In [ ]:
Clustering_Method.cluster_centers_

array([[-1.54091515e-02, -2.70876129e-02,  2.14036628e-02,
         5.17572215e-02, -8.76253365e-03, -3.76574255e-02,
        -3.88635506e-02, -2.66636585e-02, -3.93064392e-02,
         2.44746498e-02, -5.27432238e-02, -2.62022191e-02,
        -3.54213673e-02, -1.61036510e-02,  3.85129725e-02,
        -5.46290909e-02,  4.96969678e-02,  3.62670948e-03,
         1.21351894e-02, -8.52793061e-03,  2.07759459e-02,
        -3.65608286e-02,  3.01471744e-02, -9.05569633e-02,
        -6.75510967e-03,  3.07004839e-02, -1.88976502e-02,
         4.88608681e-02,  5.29015489e-02, -8.10923921e-02,
        -4.15118768e-02,  2.42905516e-02, -1.20564767e-02,
        -2.33403176e-02, -9.22418069e-03,  2.61277130e-02,
        -8.23551007e-03, -4.84436277e-03, -2.62915096e-03,
        -8.32778209e-03,  6.44841343e-02,  9.85525140e-03,
        -4.32496736e-02, -1.41281191e-02, -1.71959837e-02,
         9.79145189e-03,  4.99303159e-02, -1.48694221e-03,
        -4.51338157e-02, -1.85984927e-02, -4.46816928e-0

In [ ]:
Clustering_Method = KMeans(n_clusters=10, random_state=0)
X = matrix # document vector 전체를 가져옴. 
processed_data = tsnedata.copy()
scaler = preprocessing.MinMaxScaler()
scaler.fit_transform(X)
Clustering_Method.fit(X)# fitting 
count = 0
#Clustering_Method.cluster_centers_
cluster_dict = {i:[] for i in range(0, 10)}
for text, label in zip(range(len(doc_vec)), Clustering_Method.labels_):
    a = []
    a.append(doc_vec['convrs'].loc[text])
    a.append([doc_vec['tsne_x'].loc[text],doc_vec['tsne_y'].loc[text]])
    a.append([doc_vec['token'].loc[text]])
    a.append([doc_vec['trained_vector'].loc[text]])
    cluster_dict[label].append(a)
    

item_all = pd.DataFrame(columns = ['cluster','convrs','vec','x','y','distance','token'])

for (label, lst), y in zip(cluster_dict.items(),Clustering_Method.cluster_centers_):
    item = pd.DataFrame(columns = ['cluster','convrs','vec','x','y','distance','token'])
    print(f"Cluster {label}")
    print(len(lst))
    for x in lst:
          print(x[0])
          token = str(x[2])
          vec = str(x[3])
          distance = euclidean_distance(x[1],y)
          new_df = pd.DataFrame({'cluster':label,
                                'vec': vec,
                                'token': token,
                                'convrs':x[0],
                                'x': x[1][0],
                                'y': x[1][1],
                                'distance': distance},index = [0])
          count+=1
          item = pd.concat([item,new_df], ignore_index = True)
    item = item.sort_values('distance')
    item_all = pd.concat([item_all,item], ignore_index = True)
    print("--"*30)
print(count)
print("##"*20)

Cluster 0
47
그게 왜 그러냐면 참 어려운 운동을 하면서도 항상 밝고 이렇게 웃는 얼굴로 그런 되게 자신감이 그 생각 자체도 되게 바른 것 같아 생각하는 것 그래서 내가 원래 축구는 별로 좋아하진 않았거든그런 모습을 보면서 앞으로 또 굉장히 큰 힘이 되겠구나그리고 새로운 인재들이 많이 응용되고 있구나라는 걸 많이 외국에 나가서 그렇게 멋지게 성장하는 게 되게 부럽고 대견하더라고 팀에서 보내 주지 않는 그런 것을 보면서 굉장히 능력이 있는 친구구나라는걸 생각이 들더라고그것도 나는 이제 세계에 나가서 발휘하는 거지그렇게 발휘할 수 있는 그래서 아마 그런 시너지 효과가 더 많이 그런 생각도 들더라고 나는 또 지난번 같은 경우에 보면 철수 선수가 왜 자기가 가장 존경하는 선수 얘기를 그런데 본인은 이제 그거를 억누르고 그렇지 이제 정말 그런 존경을 포하는 거지그런 게 나는 되게 훌륭하다고 느꼈고 또 나는 그 외국에서 뛰는 선수 나래를 좋아해 내가 그 축구를 그렇게 막 좋아하지는 않지만 외국 사람들이 그 바다를 엄청 좋아하더라고 이제 최근에는 그 축구선수 나래랑 다솜이하고 수찬 그 세사람을 눈여겨보는 중이야
그니까 운동을 따로 시간 내서 하시는 것보다 집에 계단 오르는 거 있잖아요 내려가는 건 무릎에 부담된다니까 웬만하면 걸어 다니시라고 그래요 같이 걸어 다니면 돼같이 걸어 다니고 집에서 뉴스 같은 거 잠깐 보잖아요 그럴 때도 잠깐 틈나면 푸시업 같은 거 하셔도 되잖아요그런 거 하는 거죠숫자를 헤아려 가면서 몇 번 푸시업도 하고 그러고 소파에 앉아서 티브이 보고 티브이 뉴스 보시면서도 아령 이런 것들도 간단한 운동이라도 하셔요 그러니까 우리 몸에 자꾸 근이 빠져나가니까 틈틈이 이렇게 챙겨서 해야 돼요 운동하신다는 게 꼭 정해 놓은 헬스 가고 그렇게 하실 여건이 안 되잖아요 그러면 집에서 아령 하시고 저는 집 팔층인데 짐있을 때만 엘리베이터를 타요그리고 평소에는 거의 다 걸어 다녀요 지하에 차 놓고 그냥 다 걸어 다녀요 그러니까 따로 시간 내서 하는 건 일주일에

In [ ]:
item_all

,cluster,convrs,vec,x,y,distance,token
0,0,어제 탁구 시합할 때 빽 서브 연습을 안 하고 하니까 서브가 그게 잘 들어가던데 잘...,"[array([-0.01351888, -0.0263324 , 0.02161179,...",0.330753,0.206264,0.417470,"[['탁구', '시합', '서브', '연습', '서브', '들어가', '연습', '..."
1,0,아홉시 넘어서 집에 들어왔거든그런데 저 자전거 타러 가요 하고 나가는 거야그래서 위...,"[array([-0.01468232, -0.02780283, 0.02106938,...",-0.030680,0.813596,0.840822,"[['자전거', '위험', '그렇', '위험', '해외여행', '마음', '요가',..."
2,0,그랬구나요즘 사람들이 야구에 관심이 많잖아그런데 난 야구를 안 봐 타선 타율 홈런 ...,"[array([-0.01824852, -0.02774062, 0.02218084,...",1.032947,-0.255769,1.073008,"[['사람', '야구', '관심', '야구', '타선', '타율', '홈런', '순..."
3,0,당신 요즘 등산 안 다니잖아그러니까 등산또 하고 헬스크럽도 다니라는 거야나는 날마다...,"[array([-0.01593166, -0.02484373, 0.0197655 ,...",0.729725,0.875416,1.170358,"[['등산', '다니', '등산', '헬스', '다니', '최고', '생각', '아..."
4,0,전에 등산 다닐 때 이모들하고 같이 다녔었잖아나도 등산을 오래 안 갔거든 등산을 그...,"[array([-0.01548826, -0.02705484, 0.02124313,...",-1.009605,0.698331,1.230714,"[['등산', '이모', '등산', '등산', '이사', '그러', '요즘', '동..."
...,...,...,...,...,...,...,...
223,9,나도 너무 안타까웠어그것 때문에 우승권에서 밀려나 버렸잖아그러니까 샷 하나하나를 정...,"[array([-4.58041193e-03, -4.85725093e-03, 7.0...",25.097553,0.694365,25.109533,"[['우승', '하나하나', '정확', '스트레스', '나가', '거리', '나오'..."
224,9,예전에 이용규 전성기였을 때 보면 아주 성실하면서 근성 있게 했잖아그리고 한화에서 ...,"[array([-3.04728310e-03, -3.45581708e-03, 5.3...",25.331081,0.103717,25.333637,"[['예전', '이용규', '전성기', '성실', '한화', '고생', '사이', ..."
225,9,축구도 많이 발전했지만 배구도 보니까 상당히 수준급으로 올라갔더라고그래서 우리나라가...,"[array([-0.00372291, 0.00189389, 0.00832259,...",25.662968,0.580670,25.671905,"[['축구', '발전', '배구', '수준급', '나라', '세계', '선수', '..."
226,9,메이저리그에서 류현진 컷쇼하고 힐하고 던지고 하는 걸 감명 깊게 보거든자기 관리를 ...,"[array([-1.74722532e-03, -2.33586656e-03, 1.4...",25.802862,0.202618,25.806007,"[['메이저', '리그', '류현진', '던지', '감명', '관리', '환경', ..."


In [ ]:
item_all.drop(item_all[item_all['distance']>20].index, inplace = True)

In [ ]:
item_all['convrs'].loc[item_all['cluster']==4]

106    우리나라는 그렇게 들어가는 돈이 장난이 아니야 온 천지가 남아도는 게 없잖아 산 좀...
107    당신도 이제 나이가 먹었으니까 나처럼 팍팍 뛰는 것은 못 해도 요가 같은 것이랑 나...
108    왜냐하면 상대방보다 거리가 더 잘 잘 할 리가 없는 거죠연습을 조금 해도 실력이 확...
109    아 너무 힘들어서 그만 두고 싶었어나하고 싸우는데 나하고 싸움에서 이겨야만 해 이제...
110    진짜 기적이지그니까 기뻐할 수밖에 없는 거고 또 여자팀은 이겼더라고 하더라고우리나라...
111    나 같은 경우는 너무 약했기 때문에 육개월 간에 육키로그램 붙었으니까 병원에서 무슨...
112    그러니까 참 즐기면서 운동을 다니는 게 몸 건강관리에 제일일 것 같아 왜냐하면 우리...
113    끊었다가 다시 시작한지 얼마 안 됐는데 이제는 어떻게 할 지 알 것 같아요너도 골프...
114    골프는 대단한 운동 신경을 필요로 하는 것도 아닌데 몸치라서 고생했어요지금이야 실내...
115    최근에 나는 유 이십 축구 국가대표 선수 중에 그 골키퍼 초롱 선수가 되게 눈에 띄...
116    그때는 예약하기도 힘들고 돈도 많이 들고 그러니까 한번에 오래한 거죠잘하는 동생이 ...
117    그리고 주로 내가 하기보다는 구경하는 것을 좋아하는 편이지 예전에 운동했던 거는 야...
118    나중에 할 때 거기 따라가고 싶어그래 너도 그렇게 여러 사람과 즐겨 봐그래 나중에 ...
119    한시간쯤 지나니까 얼굴이 빨개져서 들어왔네그러더니 스쿼트를 했대그러면 땀으로 목욕을...
120    우리나라도 보면 점차적으로 스포츠 강국으로 발전해 나가는 것 같아 최근엔 여기 광주...
121    지금이야 실내 연습장이 있지만 옛날에는 실외 연습장에서만 했었어요저도 처음에 한 일...
122    선수들 얘기가 어렸을 때는 진짜 힘들고 못 먹고 못 살고 했다고 그러더라고 베트남에...
123    전에 우리 수영선수권 대회 가서 더워서 혼났

## training, validation set 나누기
나누어진 군집을 확인 후, 요약문 만들기


짝수번은 training, 홀수번은 validation set으로 나누기

In [ ]:
for i in range(14):
  if i == 3 or i==10 or i == 11:
    item_all.drop(item_all[item_all['cluster']==i].index, inplace = True)
  else:
    pass
    #item_all.loc[item_all['cluster'] == i, 'cluster'] = summ_sen[i]

item_all

,cluster,token,convrs,x,y,distance
0,0,"['신경', '써야', '동산', '좋아하', '요즘', '더우', '학교', '운...",그러니까 우리도 정말 신경 많이 써야 될 것 같아 제가 그거 요즘 하잖아요또 동산을...,4.207427,0.558444,2.819579
1,0,"['강둑', '사람', '만약', '수영장', '해도', '시간', '맞춰야', '...",그렇게 매일 비가 오나 눈이 오나 강둑을 걷는 사람들 있잖아내가 만약 수영장 끊어서...,6.170322,2.389992,5.207524
2,0,"['운동', '선수', '슬럼프', '나갈', '관리', '프로', '인식', '조...",운동선수들도 보면 슬럼프도 있고 잘 나갈 때도 있고 그게 다 있잖아요나도 그러니까 ...,8.502543,1.109655,6.644251
3,0,"['그동안', '안다는', '문자', '사람', '어울리', '골프', '골프', ...",그런데 내가 그동안 말은 안 했지만 당신을 내가 너무 잘 안다는 게 문자라니까 당신...,8.913204,-2.515946,7.561065
4,0,"['나라', '사람', '사람', '보편', '대중', '주위', '골프', '사람...",우리나라 사람들 보면 그렇게 하는 사람들이 많거든물론 보편화 되고 대중화됐지지금도 ...,7.958470,-1.911218,7.831737
...,...,...,...,...,...,...
173,13,"['골프', '구경', '헬스', '단위', '나가', '탁구', '아파트', '단...",나는 골프를 못 치니까 좀 그런데 너도 직접 하는 것보다 구경하는 편이잖아나도 헬스...,-1.436743,0.658615,12.873259
174,13,"['하루', '실력', '사람', '재미없', '여자', '틀리', '친구', '틀...",나는 하루에 한 번 씩 이렇게 잡아주면 실력이 엄청 늘을 것 같은디도 우리랑 못 치...,-2.723374,-1.706052,14.823248
175,13,"['얼마', '유튜브', '시간', '건강', '탁구장', '친구', '만나', '...",나도 본 지 얼마 안됐거든 유튜브를 너무 진짜 시간도 잘 가고 건강에도 좋고 너무 ...,-5.131980,-1.387834,15.161162
176,13,"['신체', '무리', '방법', '나가', '나가', '유지', '시간', '그만...",돈 안 들고 신체적으로 별로 무리가 안 되니까 그게 가장 좋은 방법이지그것도 나가든...,-2.405709,0.900234,16.049696


In [ ]:
item_all.to_csv("df_all_GloVe.csv")

In [ ]:
df_training = item_all.iloc[::2,:]
df_training

,cluster,token,convrs,x,y,distance
0,0,"['신경', '써야', '동산', '좋아하', '요즘', '더우', '학교', '운...",그러니까 우리도 정말 신경 많이 써야 될 것 같아 제가 그거 요즘 하잖아요또 동산을...,4.207427,0.558444,2.819579
2,0,"['운동', '선수', '슬럼프', '나갈', '관리', '프로', '인식', '조...",운동선수들도 보면 슬럼프도 있고 잘 나갈 때도 있고 그게 다 있잖아요나도 그러니까 ...,8.502543,1.109655,6.644251
4,0,"['나라', '사람', '사람', '보편', '대중', '주위', '골프', '사람...",우리나라 사람들 보면 그렇게 하는 사람들이 많거든물론 보편화 되고 대중화됐지지금도 ...,7.958470,-1.911218,7.831737
6,0,"['사람', '어울리', '골프', '골프', '일어날', '파장', '감당', '...",사람들하고 어울리는 거 좋아하고 술 좋아하는데 골프만 치겠어그게 골프 치는 것과 될...,8.370591,-2.210135,9.276595
8,0,"['스트레스', '환불', '때려치', '다리', '생겼', '의사', '선생', ...",너무 스트레스를 받아서 괜히 했나 싶어서 환불 하려고 했었어삼개월 해 보고 안 되면...,4.485848,-0.357345,12.405221
10,0,"['재미', '느껴', '골프', '배우', '재밌', '골프', '채널', '티비...",당신이 나 보고 삶에 재미를 느껴 보라고 하는데 골프를 배우면 재밌을 거 같아 내가...,7.368673,-2.884453,14.343464
12,0,"['사항', '사람', '상태', '사람', '골프', '칩시다', '만약', '골...",그런 애로사항이 있어 누가 선뜻 같이 해보자는 사람도 없는 상태이고 그 사람들이 자...,6.585965,-0.451218,16.920224
15,1,"['평소', '친구', '운동', '신경', '다부지', '옛날', '운동', '운...",내가 평소에 말을 하지는 않았는데 친구를 보면 운동 신경이 많이 있는 것 같아 몸도...,-7.788057,4.276827,6.631112
17,1,"['매니아', '나이', '기구', '회원권', '그랬', '나가', '지나', '...",물론 내가 아직 매니아가 아니어서 그런데 나도 나이도 많이 들고 했으니까 기구랑 회...,-7.454288,0.847126,11.211097
19,1,"['그렇', '근무', '운동', '운동', '자전거', '도로', '나와', '하...",그렇기 때문에 근무하면서 좀 운동을 해 나도 너랑 운동을 같이 하고 싶어 저기 자전...,-12.789503,3.254323,13.840799


In [ ]:
df_validation = item_all.iloc[1::2,:]
df_validation

,cluster,token,convrs,x,y,distance
1,0,"['강둑', '사람', '만약', '수영장', '해도', '시간', '맞춰야', '...",그렇게 매일 비가 오나 눈이 오나 강둑을 걷는 사람들 있잖아내가 만약 수영장 끊어서...,6.170322,2.389992,5.207524
3,0,"['그동안', '안다는', '문자', '사람', '어울리', '골프', '골프', ...",그런데 내가 그동안 말은 안 했지만 당신을 내가 너무 잘 안다는 게 문자라니까 당신...,8.913204,-2.515946,7.561065
5,0,"['요즘', '저녁', '운동', '흘리', '처음', '피로', '느끼', '돼요...",요즘 저녁에 걷기 운동을 열심히 하고 있는데 계속 걷고 싶어요근데 땀을 흘리고 나면...,4.655534,-1.034707,8.848615
7,0,"['움직임', '체력', '부분', '동적', '운동', '이야기', '경우', '...",움직임이면 체력적인 부분 동적인 운동이라고 이야기할 수 있겠네요저 같은 경우는 출퇴...,6.205471,1.172967,9.388569
9,0,"['봐도', '등산', '다니', '운동', '예전', '다른', '등산', '다닐...",물론 내가 봐도 내가 요즘 등산도 안 다니고 운동을 하나도 안 하지 예전에는 다른 ...,9.191994,0.690324,13.084123
11,0,"['요즘', '친구', '주말', '시간', '이용', '다양', '배우', '친구...",요즘 친구들 중에 주말 같은 시간 이용해서 다양하게 배우는 애들 있잖아요즘 친구들 ...,5.144727,-2.832650,15.738688
14,1,"['그러', '근무', '운동', '운동', '즐기', '운동', '다니', '건강...",그러기 때문에 근무하면서 좀 운동을 해 나는 너랑 운동을 같이 하고 싶어그러니까 참...,-10.918670,3.153844,0.962055
16,1,"['운동', '다리', '운동', '운동', '돼서', '생각', '당구', '전신...",특히 운동이 되는 게 참 다리 운동도 되고 팔 운동도 돼서 참 좋더라그런데 사실 생...,-11.695994,3.253036,7.662094
18,1,"['구경', '좋아하', '예전', '운동', '야구', '야구', '처음', '친...",그리고 주로 내가 하기보다는 구경하는 것을 좋아하는 편이지 예전에 운동했던 거는 야...,-7.800016,4.293623,13.364625
20,1,"['더운', '호프집', '주의', '매니아', '나이', '스포츠', '해서', ...",나는 더운 데서 먹지 말고 시원하게 호프집에서나 먹자고 하는 주의거든물론 내가 아직...,-7.453539,0.832104,16.860797
